## Assignment 2 - Cryptographic Systems and Protocols Group Assignment
### Megha Sapra, Odyssey Villagomez


Part 1 - Using any programming language (Preferably Java or Python), develop your own hash function. Given an input string x of any length, your hash function must generate a random output bit vector y of length 32 bits. You should only use logical operators (&, |, >>, <<) and also (Rotate right/left) to generate the output (digest). You can take ideas from how existing Hash functions like SHA-2 are designed. 

In [255]:
import string 
import random

In [288]:
# rotate functions from https://www.geeksforgeeks.org/python-logical-operators-with-examples-improvement-needed/
def leftrotate(s, d):
    tmp = s[d : ] + s[0 : d]
    return int(tmp)
def rightRotate(n, d):
    return leftrotate(s, len(s) - d)

In [289]:
# Binary operations
def F(X,Y,Z):
    return( (X&Y)|((~X)&Z) )

def FF(a,b,c,d,M,s):
    result = b + leftrotate( str(a+F(b,c,d)+M), s)
    return(result)

def hashFunction(x):    #generate random output of length 32 bits
    A = 0x67452301
    B = 0xefcdab89
    C = 0x98badcfe
    D = 0x10325476
    
    x_bytes = x.encode()     #enocde string to bytes 
    hashArr = []
    
    # for each 'block' in the string, perform some operations
    # block is 8 bits, 
    
    for block in x_bytes: 
        #Each block is further broken in 2 bit words

        block = bin(block)    #convert block to its corresponding bits 
        bits = block[2:len(block)]    #only keep actual bits, not starting '0b' value 

        # pad block to size 8 
        if len(bits) < 8: 
            for i in range (len(bits),8):
                bits = bits + '1'

        word1 = bits[0:2]
        word2 = bits[2:4]
        word3 = bits[4:6]
        word4 = bits[6:8]
        
        #perform logical operations sub blocks  (words)
        shift = random.randint(1,10)
        a = FF(A,B,C,D, int(word1),shift)
        
        shift = random.randint(1,10)
        b = FF(A,B,C,D, int(word2),shift)
        
        shift = random.randint(1,10)
        c = FF(A,B,C,D, int(word3),shift)
        
        shift = random.randint(1,10)
        d = FF(A,B,C,D, int(word4),shift)
            
        #combine outputs and add to hash array
        new_byte = a + b + c + d
        hashArr.append(new_byte)
        
        
    
    hashValue = ''
    for item in hashArr:     #combine values from the blocks
        hashValue = hashValue + str(bin(item)) #convert the integer to its binary form so that we can limit to 32 bits
        
    return str(hashValue)[0:32] #limit hash digest to 32 bits 

Part 2 - Then, write a method that finds collisions in your hash function using brute-force attack. This method will generate many random input strings and stops when two strings have the same hash. Analyze your hash function in terms of robustness using the concept of birthday attack.

In [282]:
def bruteForceAttack():
    print("Attacking...")
    counter = 0; 
    noCollision = True
    length = random.randint(1,1000)
    while(noCollision):
        randomString1 = (''.join(random.choices(string.hexdigits, k= 10 )))    #generted randomly 
        randomString2 = (''.join(random.choices(string.hexdigits, k= 10)))     #generted randomly
        
        hash1 = hashFunction(randomString1)
        hash2 = hashFunction(randomString2)
        
        if hash1 == hash2:
            #print(randomString1, randomString2)
            print("collison at", hash1, "equals", hash2)
            noCollision = None    #stop while loop
            return counter
        else:
            counter = counter + 1 
        
        if counter >= 65536:     #(2^16 tries)
            noCollision = None
            return counter

In [287]:
total = 0
for i in range(0,5):
    attempts = bruteForceAttack()
    total = total + attempts


print("Average attempts to get collision: ", total/5)

Attacking...
collison at 0b110000100010000111111010010100 equals 0b110000100010000111111010010100
Attacking...
collison at 0b100101111100101000011111101101 equals 0b100101111100101000011111101101
Attacking...
Attacking...
collison at 0b100100100010011111101011011110 equals 0b100100100010011111101011011110
Attacking...
collison at 0b111101001011011001001001010000 equals 0b111101001011011001001001010000
Average attempts to get collision:  22735.0


Part 3 - Write a short report describing your hash function, your brute-force attack function, and your analysis on its robustness.

The hash function is modeled after SHA-256 where the input string s is divided into blocks (size 8 bits) and then those blocks are further divided into sub blocks (size 2 bits). The sub block performs binary operations on the word, combining it with inialization values. The result of the sub blocks are concatenated and only the first 32 bits of the value are kept, to output a digest of size 32 bits. 

For the hash function to be secure it needs to be resitant to (2^16) brute force attacks, defined by the birthday bound of 2^(n/32) where n is the size of the hash, 32. 

The hash funciton was evaluated by attempting a brute force attack 5 times. The amount of steps to break the attack are averaged and result in average steps of ~15,000, ~7000, and 22000. This hash function is not secure, based on the birthday attack because it takes less than 2^16 steps to find a collision